In [2]:
import pandas as pd
import numpy as np
import re

# mountain station

http://www2.nsysu.edu.tw/Mountain/trick/coordinates.htm

In [8]:
mou_info = pd.read_csv('mou-station-info')
mou_info = mou_info[mou_info.SiteName.isin(['牧山','大霸尖山','玉山','北大武山'])]
mou_info['SiteEngName'] = ['Yushan','Dabajianshan','Mushan','Beidawushan']
mou_info = mou_info[['SiteName','SiteEngName','lat','lon']]
mou_info['area'] = 'mountain'
mou_info

,SiteName,SiteEngName,lat,lon,area
0,玉山,Yushan,23.470556,120.948889,mountain
26,大霸尖山,Dabajianshan,24.461111,121.249722,mountain
66,牧山,Mushan,23.864722,121.152222,mountain
92,北大武山,Beidawushan,22.629167,120.751944,mountain


# epa station

In [9]:
epa_info = pd.read_csv('epa-station-info')
epa_info['area'] = 'city'
epa_info.head()

,SiteName,SiteEngName,AreaName,County,Township,SiteAddress,lon,lat,SiteType,lat_round,lon_round,row,col,area
0,陽明,Yangming,北部空品區,臺北市,北投區,臺北市北投區竹子湖路111號,121.529583,25.182722,公園測站,25.18,121.53,16,153,city
1,萬里,Wanli,北部空品區,新北市,萬里區,新北市萬里區瑪鋉路221號,121.689881,25.179667,一般測站,25.18,121.69,16,169,city
2,淡水,Tamsui,北部空品區,新北市,淡水區,新北市淡水區中正東路42巷6號,121.449239,25.164500,一般測站,25.16,121.45,18,145,city
3,基隆,Keelung,北部空品區,基隆市,信義區,基隆市信義區東信路324號,121.760056,25.129167,一般測站,25.13,121.76,21,176,city
4,士林,Shilin,北部空品區,臺北市,北投區,臺北市北投區文林北路155號,121.514500,25.105917,一般測站,25.11,121.51,23,151,city


# combine station

In [10]:
import os
import sys
home=os.path.expanduser("~")
sys.path.append(os.path.join(home, 'station2grid'))
from tools.knn_model import KnnModel
from tools.options import *
from tools import datasets

In [11]:
optionS2GSD = OptionS2GSD()
dataS2G = datasets.DataS2G(optionS2GSD)
model = KnnModel(dataS2G)

In [12]:
sta_info = epa_info.append(mou_info, sort=False)
sta_info['lat_round'] = sta_info.lat.apply(lambda x: round(x,2))
sta_info['lon_round'] = sta_info.lon.apply(lambda x: round(x,2))
sta_info.loc[67, 'lat_round'] = 22.60 # 前鎮

sta_info['row'] = sta_info.lat_round.apply(lambda x: [round(y,2) for y in model.lats].index(x))
sta_info['col'] = sta_info.lon_round.apply(lambda x: [round(y,2) for y in model.lons].index(x))

sta_info = sta_info.sort_values(by=['area','lat',],ascending=False).reset_index(drop=True)
sta_info = sta_info[['SiteName',
                     'SiteEngName',
                     'area',
                     'lat',
                     'lon',
                     'lat_round',
                     'lon_round',
                     'row',
                     'col']]
sta_info


,SiteName,SiteEngName,area,lat,lon,lat_round,lon_round,row,col
0,大霸尖山,Dabajianshan,mountain,24.461111,121.249722,24.46,121.25,88,125
1,牧山,Mushan,mountain,23.864722,121.152222,23.86,121.15,148,115
2,玉山,Yushan,mountain,23.470556,120.948889,23.47,120.95,187,95
3,北大武山,Beidawushan,mountain,22.629167,120.751944,22.63,120.75,271,75
4,陽明,Yangming,city,25.182722,121.529583,25.18,121.53,16,153
5,萬里,Wanli,city,25.179667,121.689881,25.18,121.69,16,169
6,淡水,Tamsui,city,25.164500,121.449239,25.16,121.45,18,145
7,基隆,Keelung,city,25.129167,121.760056,25.13,121.76,21,176
8,士林,Shilin,city,25.105917,121.514500,25.11,121.51,23,151
9,林口,Linkou,city,25.078570,121.365703,25.08,121.37,26,137


In [13]:
sta_info.to_csv('station-info',index=False)

In [14]:
pd.read_csv('station-info')

,SiteName,SiteEngName,area,lat,lon,lat_round,lon_round,row,col
0,大霸尖山,Dabajianshan,mountain,24.461111,121.249722,24.46,121.25,88,125
1,牧山,Mushan,mountain,23.864722,121.152222,23.86,121.15,148,115
2,玉山,Yushan,mountain,23.470556,120.948889,23.47,120.95,187,95
3,北大武山,Beidawushan,mountain,22.629167,120.751944,22.63,120.75,271,75
4,陽明,Yangming,city,25.182722,121.529583,25.18,121.53,16,153
5,萬里,Wanli,city,25.179667,121.689881,25.18,121.69,16,169
6,淡水,Tamsui,city,25.164500,121.449239,25.16,121.45,18,145
7,基隆,Keelung,city,25.129167,121.760056,25.13,121.76,21,176
8,士林,Shilin,city,25.105917,121.514500,25.11,121.51,23,151
9,林口,Linkou,city,25.078570,121.365703,25.08,121.37,26,137
